In [1]:
import pandas as pd
import numpy as np
import hdf5_getters as hd
import glob
import sklearn
from sklearn import preprocessing

In [2]:
#To see if all the 10,000 files are available for data extraction
h5_dir = "MillionSongSubset" #<file_path where the song_dataset is stored>

# Get a list of all .h5 files in the directory and its subdirectories
h5_files = glob.glob(h5_dir + '/**/*.h5', recursive=True)
len(h5_files) #should be 10,000

10000

In [3]:
#Reading the dataset
df = pd.read_csv("3593_song_newfeatures_genre_dataset.csv")

In [4]:
#Filtering out data for Country genre 
df_song = df[df['genre'] == 'Country']
df_song

,Unnamed: 0,id,spectral_centroid_mean,spectral_centroid_dev,rolloff_point_mean,rolloff_point_dev,flux_mean,flux_dev,variability_mean,variability_dev,compactness_mean,compactness_dev,rms_mean,rms_dev,low_energy_window_mean,low_energy_window_dev,zero_crossing_mean,zero_crossing_dev,title,genre
2,2,TRBGMAW128F4231326,10.680,6.049,0.06769,0.04254,0.000315,0.000589,0.002009,0.001721,1735.0,265.2,0.07812,0.06550,0.5651,0.08847,26.56,14.110,b'Terraplane Blues',Country
49,49,TRBGPWE128F421CCB4,7.670,7.364,0.05100,0.05439,0.002256,0.002410,0.004360,0.001985,1718.0,257.7,0.16590,0.06970,0.5331,0.07067,22.22,17.750,b'All For The Sake Of The Song',Country
77,77,TRBGSCJ128F92F6D47,8.855,8.323,0.05596,0.05242,0.001044,0.000963,0.004369,0.001647,1684.0,313.9,0.16830,0.06070,0.4822,0.07439,22.94,17.570,"b""I'll Be Around""",Country
117,117,TRBHCQA128F149F192,11.900,9.723,0.08892,0.07364,0.003367,0.002751,0.005584,0.001798,1643.0,201.9,0.21660,0.06101,0.5271,0.05337,34.27,21.820,b'She Runs',Country
124,124,TRBHFDA128F4270728,8.991,7.841,0.06113,0.05882,0.000778,0.002056,0.002103,0.001508,1644.0,241.9,0.08071,0.05250,0.6153,0.07925,25.83,17.790,b'Slow Hand (LP Version)',Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550,3550,TRAYLQT128F425A4A6,6.165,2.915,0.03815,0.02191,0.000438,0.000837,0.002340,0.001254,1721.0,261.0,0.08704,0.04229,0.5404,0.09075,14.72,6.963,"b""God's Hands""",Country
3554,3554,TRAYEIJ128F42A53E0,13.980,7.606,0.10330,0.07605,0.002883,0.005564,0.004624,0.002210,1608.0,175.8,0.18130,0.07743,0.6114,0.08036,45.78,18.690,b'I Just Knew',Country
3573,3573,TRAYDYV128F42906E1,6.892,6.995,0.04579,0.06566,0.002035,0.002830,0.004325,0.001825,1680.0,187.5,0.16140,0.06215,0.5470,0.06465,24.28,18.630,"b""Breakin' It""",Country
3577,3577,TRAYXZF128F425D645,13.940,8.201,0.10330,0.07512,0.001944,0.001798,0.004671,0.001986,1591.0,169.8,0.18550,0.07380,0.4716,0.05560,40.36,22.150,b'A Boy Named Sue',Country


In [5]:
#Creating lists of Genre and Track_id
track_id = df_song['id'].tolist()
genre = df_song['genre'].tolist()

In [6]:
genre

['Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Country',
 'Co

In [7]:
#Function to extract the cleaned track_id
def clean(text):
    text = str(text)
    text = text.replace("b'", "")
    text = text[:-1]
    return text

In [8]:
#Collecting segment info for Loudness only for those songs in the 3953_dataset
batch_size = 10

song_id = []
loud_max_seg = []
pitch_seg = []
timbre_seg = []

for i in range(0, len(h5_files), batch_size):
    # Get the batch of files
    batch_files = h5_files[i:i+batch_size]
    
    # Loop through the files in the batch
    for file_name in batch_files:
        h5 = hd.open_h5_file_read(file_name)
        
        id = hd.get_track_id(h5)
        id = clean(id)

        if id in track_id:
            #print("Collecting data")
            song_id.append(hd.get_track_id(h5))
            loud_max_seg.append(hd.get_segments_loudness_max(h5))
            timbre_seg.append(hd.get_segments_timbre(h5))
            pitch_seg.append(hd.get_segments_pitches(h5))
        
        h5.close()

In [9]:
#All are equal to the number of songs filtered out from the Country genre
len(song_id), len(loud_max_seg), len(timbre_seg), len(pitch_seg), len(genre)

(174, 174, 174, 174, 174)

In [10]:
#Inspecting values of loudness for different segments for 2 songs.
loud_max_seg[0:2]

[array([-60.   , -18.708, -20.249, -22.201, -14.352, -19.434, -20.166,
        -21.356, -20.423, -17.625, -20.317, -22.631, -21.78 , -17.76 ,
        -20.673, -18.606, -20.869, -17.696, -18.908, -17.789, -19.405,
        -16.606, -17.691, -20.94 , -16.644, -18.144, -16.71 , -21.386,
        -17.844, -14.969, -14.461, -13.693, -13.888, -13.183, -11.042,
        -16.676, -13.715, -14.481, -14.069, -14.392, -18.812, -12.976,
        -17.313, -15.649, -13.306, -17.493, -17.914, -14.885, -21.209,
        -12.189, -16.024, -13.614, -15.504, -16.961, -17.83 , -17.015,
        -14.471, -19.388, -12.005, -16.64 , -16.679, -15.117, -12.609,
        -13.888, -16.807, -19.318, -12.562, -17.295, -17.502, -14.96 ,
        -14.138, -16.83 , -17.68 , -16.007, -18.771, -20.829, -13.773,
        -14.567, -17.729, -17.249, -11.583, -11.227, -19.044, -18.58 ,
        -12.37 , -14.772, -10.729, -14.63 , -13.177, -13.08 , -10.235,
        -13.187, -11.296, -15.185, -10.869, -13.545, -10.593,  -7.628,
      

In [11]:
#inspecting the segments' shape for all songs
for arr in loud_max_seg:
    print(arr.shape)

(624,)
(659,)
(535,)
(606,)
(534,)
(417,)
(784,)
(115,)
(297,)
(603,)
(531,)
(266,)
(785,)
(849,)
(640,)
(706,)
(566,)
(597,)
(638,)
(557,)
(686,)
(518,)
(823,)
(392,)
(469,)
(712,)
(741,)
(512,)
(647,)
(587,)
(407,)
(937,)
(305,)
(499,)
(502,)
(737,)
(418,)
(474,)
(517,)
(632,)
(554,)
(459,)
(585,)
(731,)
(744,)
(602,)
(530,)
(459,)
(707,)
(851,)
(526,)
(686,)
(421,)
(798,)
(373,)
(525,)
(625,)
(654,)
(621,)
(509,)
(428,)
(698,)
(495,)
(422,)
(387,)
(921,)
(386,)
(891,)
(494,)
(671,)
(642,)
(598,)
(486,)
(443,)
(737,)
(805,)
(556,)
(603,)
(596,)
(529,)
(707,)
(667,)
(561,)
(465,)
(829,)
(511,)
(887,)
(430,)
(741,)
(978,)
(596,)
(502,)
(683,)
(961,)
(482,)
(693,)
(742,)
(1259,)
(352,)
(120,)
(771,)
(369,)
(467,)
(590,)
(640,)
(715,)
(387,)
(726,)
(738,)
(600,)
(908,)
(784,)
(835,)
(464,)
(822,)
(775,)
(987,)
(659,)
(1046,)
(530,)
(619,)
(631,)
(688,)
(624,)
(478,)
(295,)
(474,)
(544,)
(857,)
(830,)
(536,)
(480,)
(471,)
(556,)
(793,)
(939,)
(704,)
(450,)
(738,)
(517,)
(523,)
(303,)
(885

In [12]:
#Installing Librosa Library for creating spectrogram
!pip install librosa
import librosa
import librosa.display
import numpy as np

In [13]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Padding the loudness arrays to be of the same length 
max_length = max(len(arr) for arr in loud_max_seg)
padded_loud_max_seg = []
for arr in loud_max_seg:
    padded_arr = np.pad(arr, (0, max_length - len(arr)), mode='constant')
    padded_loud_max_seg.append(padded_arr)
concatenated_loudness = np.vstack(padded_loud_max_seg)

# Standardization (Z-score normalization)
mean_val = np.mean(concatenated_loudness)
std_val = np.std(concatenated_loudness)
scaled_loudness = (concatenated_loudness - mean_val) / std_val

sr= 22050

# Computing the spectrogram using librosa
spec = librosa.feature.melspectrogram(y=None, sr=sr, S=scaled_loudness, n_fft=2048, hop_length=512, n_mels=64)

# Converting the power spectrogram to dB scale
spec_db = librosa.power_to_db(spec, ref=np.max)

# Plotting the spectrogram
plt.figure(figsize=(10, 5))
librosa.display.specshow(spec_db, x_axis='time', y_axis='mel', sr=sr, hop_length=512, cmap='coolwarm')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram for Country genre')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()